Tensorboard, salvataggio e loading di un modello, salvataggio loading checkpoints.
Presumibilmente sarebbe utile che per ogni training che si fa si salvi lo stato per tensorboard e si salvi anche il modello in modo da poterlo riutilizzare. Qua si trovano tutte le inoformazioni per questo scopo.

## TensorBoard

Per utilizzare tensorboard è necessario utilizzare keras all'interno di tensoflow. La sintassi non cambia, cambiano solo gli import

In [93]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Flatten, AveragePooling1D, LSTM
from tensorflow.keras.layers import Dropout, Dense, Activation
from tensorflow.keras.losses import binary_crossentropy, categorical_crossentropy
from tensorflow.python.keras.callbacks import TensorBoard
import tensorflow as tf
import numpy as np

Instanzioamo un semplice modello una mlp classica

In [94]:
x_train=np.array([[1,2,3],[7,8,9],[1,2,3]])
y_train=np.array([1,0,1])
x_val=np.array([[1,2,3],[7,8,9],[1,2,3]])
y_val=np.array([1,0,1])

In [95]:
model = Sequential()
model.add(Dense(10, activation='relu'))
model.add(Dense(10, activation='sigmoid'))
model.add(Dense(1, activation='sigmoid'))

In [96]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Ora creaiamo il callback per tensorboad. Con os.path.join si crea la directory(Ogni volta che si esegue crea una sottocartella con il timestamp). Dopodichè si crea il callback.

In [97]:
import datetime
import os
log_dir = os.path.join("..\logsTB\\"+ datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

Quando si fitta il modello si deve inserire anche il callback generato prima.

In [98]:

model.fit(x_train, y_train,
          batch_size=1,  
          epochs=20,
          validation_data=(x_val, y_val),
          verbose=True,
          callbacks=[tensorboard_callback])

Train on 3 samples, validate on 3 samples
Epoch 1/20
3/3 [==============================] - 1s 178ms/sample - loss: 0.6964 - accuracy: 0.6667 - val_loss: 0.6881 - val_accuracy: 0.6667
Epoch 2/20
3/3 [==============================] - 0s 10ms/sample - loss: 0.6864 - accuracy: 0.6667 - val_loss: 0.6800 - val_accuracy: 0.6667
Epoch 3/20
3/3 [==============================] - 0s 13ms/sample - loss: 0.6810 - accuracy: 0.6667 - val_loss: 0.6711 - val_accuracy: 0.6667
Epoch 4/20
3/3 [==============================] - 0s 8ms/sample - loss: 0.6691 - accuracy: 0.6667 - val_loss: 0.6636 - val_accuracy: 0.6667
Epoch 5/20
3/3 [==============================] - 0s 10ms/sample - loss: 0.6596 - accuracy: 0.6667 - val_loss: 0.6572 - val_accuracy: 0.6667
Epoch 6/20
3/3 [==============================] - 0s 14ms/sample - loss: 0.6551 - accuracy: 0.6667 - val_loss: 0.6498 - val_accuracy: 0.6667
Epoch 7/20
3/3 [==============================] - 0s 13ms/sample - loss: 0.6478 - accuracy: 0.6667 - val_loss: 0

Per visualizzare la tensorboard di un modello bisogna copiare il path della sua cartella (è possibile selezionare anche una cartella che contiene più di un modello). Sul prompt dei comandi digitare 

*tensorboard --logdir _path_selezionato_*

In seguito andare su localhost:6006 per avviare la visualizzazione

## Salvataggio e caricamento di un modello

Per il salvataggio e il caricamento di un modello:

In [102]:
model.save('my_model.h5') 
new_model = tf.keras.models.load_model('my_model.h5')

## Salvataggio e loading ceckpoint

è possibile effettuare il salvataggio ogni tot epoche utile per prendere l'epoca in cui il validation è minimo per esempio

In [109]:
checkpoint_path = "my_model/model-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

In questo caso facciamo ogni 5 epoche

In [110]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    period=5)

In [111]:
model.save_weights(checkpoint_path.format(epoch=0))
model.fit(x_train, 
          y_train,  
          epochs=10,
          validation_data=(x_val,y_val),
          callbacks=[cp_callback])

Train on 3 samples, validate on 3 samples
Epoch 1/10
3/3 [==============================] - 0s 5ms/sample - loss: 0.5342 - accuracy: 0.6667 - val_loss: 0.5326 - val_accuracy: 0.6667
Epoch 2/10
3/3 [==============================] - 0s 5ms/sample - loss: 0.5326 - accuracy: 0.6667 - val_loss: 0.5310 - val_accuracy: 0.6667
Epoch 3/10
3/3 [==============================] - 0s 5ms/sample - loss: 0.5310 - accuracy: 0.6667 - val_loss: 0.5294 - val_accuracy: 0.6667
Epoch 4/10
3/3 [==============================] - 0s 5ms/sample - loss: 0.5294 - accuracy: 0.6667 - val_loss: 0.5278 - val_accuracy: 0.6667
Epoch 5/10

Epoch 00005: saving model to my_model/model-0005.ckpt
3/3 [==============================] - 0s 15ms/sample - loss: 0.5278 - accuracy: 0.6667 - val_loss: 0.5262 - val_accuracy: 0.6667
Epoch 6/10
3/3 [==============================] - 0s 4ms/sample - loss: 0.5262 - accuracy: 0.6667 - val_loss: 0.5246 - val_accuracy: 0.6667
Epoch 7/10
3/3 [==============================] - 0s 5ms/sampl

Questo è per prendere un determinato checkpoint

In [117]:
model.load_weights('my_model/model-0000.ckpt')